In [ ]:
!pip install git+https://github.com/adriengoeller/python-sensor-simulation --force-reinstall

In [ ]:
# Noms:
# Prénoms :
# Classe :

In [ ]:
# Imports
from sensorsim import instruments, montages
import numpy as np

In [ ]:
# Programmation de l'enceinte

# ___________________________________
# Durée de l'expérience temps : 60 sec pas de temps 0.1s
duree_s = 60
pas_de_temps_experience_ms = 100
pas_de_temps_realite_ms = 10

tg = instruments.TimeGenerator(duree_s, pas_de_temps_experience_ms, pas_de_temps_realite_ms)

# Aide profils de pression/température
# - np.linspace(101325.0,26442.18910734488 , tg.size_exp) permet d'avoir une fonction linéaire entre le départ et l'arrivée avec tg.size_exp nombre de point
# - tg.size_exp correspond au nombre de points attendus selon les paramètres de temps plus haut.
# - On peut faire des échelons : np.ones(tg.size_exp)*20 est une valeur constante de 20 sur le nombre de point tg.size_exp
# - On peut faire plusieurs échelons : np.concatenate([np.ones(10)*0, np.ones(tg.size_exp-10)*20])
# permet d'avoir un échelon à 0 pendant 10 pas de temps et le reste à 20.
# - On peut combiner plusieurs : np.concatenate([np.ones(10)*101325.0, np.linspace(101325.0,26442.18910734488 , tg.size_exp-20), np.ones(10)*0])

# - Pression :
# la pression est paramétrée pour aller de p1 à p2
p1 = 101325.0
p2 = 26442.2
simu_P = np.linspace(p1 ,p2 , tg.size_exp)

# - Température :
simu_T = np.ones(tg.size_exp)*20

# Initialisation de l'environnement :
env = instruments.Environment(tg)



In [ ]:
# Variables :
L= 2.8e-3
e= 1e-5
E = 130e9
Igz = e*L/12*(e**2+L**2)
P_atm = 1.2e5
P = 1.013e5

# initialisation des instruments

TE = instruments.TestEnclosure(env, simu_P, simu_T, tg)
membrane = instruments.Membrane(E=env, EIgz=E*Igz, diameter=L, P_calib=P_atm)

quality="normal"
R1 = instruments.Resistance(E=env, R=10e3, quality=quality)
R2 = instruments.Resistance(E=env, R=17.9e3, quality=quality)
R3 = instruments.Resistance(E=env, R=1e3, quality=quality)
R4 = instruments.Resistance(E=env, R=1.79e3,K=2, quality=quality)


# on attache la résistance à la membrane
R4.attach_membrane(membrane,-1)


# Generateur
gen = instruments.Generateur(v_alim=5.0)

 # Resistances gain

R1_ao = instruments.Resistance(E=env, R=2.7e3)
R2_ao = instruments.Resistance(E=env, R=92e3)
R3_ao = instruments.Resistance(E=env, R=10)
R4_ao = instruments.Resistance(E=env, R=100e3)

gen_ao = instruments.Generateur(v_alim=89/250)

wheastone = montages.Wheastone(R1, R2, R3, R4)

# Etage du CAN
h = instruments.Horloge(env, 10, 10)
EB = instruments.EchantillonneurBloqueur(h,10)
can = instruments.CanCompare(h, 8, 5) # 8 bits / 5V alim

# CPU
cpu = instruments.Cpu(env,h)

# init recorder

r = instruments.Recorder(env)
r.config(
    {
        "P":["P",False],
        "T":["T",False],
        "dL":["dL",False],
        "R1":["R1",False],
        "R2":["R2",False],
        "R3":["R3",False],
        "R4":["R4",False],
        "Vm":["Vm", False],
        "Vg":["Vg", False],
        "hh":["horloge",True],
        "EB":["EB",True],
        "cout":["can output",True],
        "cint":["can internal",True],
        "cpu":["cpu",True]
    }
)


In [ ]:
# Lancement de l'expérience :

for tt in tg:

    Vm = wheastone(gen())
    Vg = montages.montage_ampli_op(gen_ao(), Vm, R1_ao(), R2_ao(), R3_ao(), R4_ao())
    vb=EB(Vg)
    can(vb)
    cpu(can.out)

    snp = {
            "P":env.P,
            "T":env.T,
            "dL":membrane.L_def,
            "R1":R1(),
            "R2":R2(),
            "R3":R3(),
            "R4":R4(),
            "Vm":Vm,
            "Vg":Vg,
            "hh":h.value,
            "EB":vb,
            "cout":can.out,
            "cint":can.internal,
            "cpu":cpu.altitude
            }

    r.snapshot(snp)

In [ ]:
# Affichage des résultats sur les appareils :
r.plot(
        {
            1:"P",2:"T", 3:"dL", 4:["R1","R2","R3","R4"], 5:"Vm", 6:"Vg",7:["cint","cout","EB"], 8:"hh",9:"cpu"
        }
    )